In [1]:
import pandas as pd
import numpy as np
import pickle
import json

In [2]:
with open('counterfact/pop-results-curie.pickle', 'rb') as handle:
    res = pickle.load(handle)

json_data = [] # your list with json objects (dicts)

with open('counterfact/counterfact.json') as json_file:
   json_data = json.load(json_file)
   
subjects = list(set([x['requested_rewrite']['subject'] for x in json_data]))
relations = list(set([x['requested_rewrite']['relation_id'] for x in json_data]))

In [247]:
def get_item(subject, relation_id=None, get_first = True):
    # out = [i for i in res[relation_id] if i['requested_rewrite']['subject']==subject]
    if relation_id!=None:
        out = [x for x in json_data if x['requested_rewrite']['subject']==subject and x['requested_rewrite']['relation_id']==relation_id]
    else:
        out = [x for x in json_data if x['requested_rewrite']['subject']==subject]
    if get_first:
        return(out[0])
    else:
        return(out)

def print_item(item):
    case_id = item["case_id"]
    subj = item["requested_rewrite"]["subject"]
    prompt = item["requested_rewrite"]["prompt"]
    targ_new = item["requested_rewrite"]["target_new"]['str']
    targ_true = item["requested_rewrite"]["target_true"]['str']
    gen_prompts = item["generation_prompts"]

    print("case:", case_id)
    print("subject:", subj)
    print("prompt:", prompt)
    print(targ_true, "-->", targ_new)
    print("------ generation prompts ------")
    print("\n".join(gen_prompts))



In [212]:
dlist = []
for r in relations:
    d = res[r].sort_values("logprob", ascending=False).head(20).reset_index()
    items = [get_item(x,r) for x in list(d.subject)]
    case_ids = [x["case_id"] for x in items]
    d["relation_id"] = r
    d["case_id"] = case_ids
    dlist.append(d)

df = pd.concat(dlist)
df.to_csv("counterfact/candidate-items.csv")
df


,index,subject,logprob,relation_id,case_id
0,294,Saint Valentine,-12.320089,P39,11025
1,114,Rob Ford,-12.487076,P39,11012
2,364,Achille Lauro,-14.088795,P39,3554
3,140,John Paul II,-14.382720,P39,799
4,174,John Calvin,-15.040618,P39,18147
...,...,...,...,...,...
15,256,Jackie Wilson,-17.008402,P19,20536
16,58,Konrad Adenauer,-17.191958,P19,10356
17,365,Brendan Behan,-17.221687,P19,3589
18,262,Aisha,-17.271600,P19,20983


In [225]:
r = relations[1]
r = "P740"
items = res[r].sort_values("logprob", ascending=False).head(100).reset_index()
i = 0
print(r)

P740


In [244]:
### for scrolling through
subj = items.loc[i]["subject"]
print_item(get_item(subj, r))
i+=1

case: 9053
subject: Lady Antebellum
prompt: {} was started in
Nashville --> Detroit
------ generation prompts ------
The founders of Lady Antebellum are from the location of
The inspiration for Lady Antebellum came from
The founders of Lady Antebellum are from the location of
The founders of Lady Antebellum are from the location of
Lady Antebellum originated from
The founders of Lady Antebellum are from the location of
The founders of Lady Antebellum are from the location of
The founders of Lady Antebellum are from the location of
Lady Antebellum originated from
Lady Antebellum originated from


In [175]:
i-=2
r

'P495'

In [942]:
chosen = pd.read_csv("counterfact/selected-items.csv")

chosen

,subject,logprob,relation_id,case_id,Include,past
0,Clark Gable,-13.349388,P1412,8597,1,1
1,Mother Teresa,-8.899356,P1412,10996,1,1
2,Jesus Christ,-8.906232,P1412,463,1,1
3,Muhammad Ali,-9.665276,P1412,7559,1,1
4,Andrew Carnegie,-11.644358,P1412,873,1,1
...,...,...,...,...,...,...
188,Siri,-9.501763,P178,21668,1,0
189,iTunes,-10.425485,P178,15438,1,0
190,Skype,-11.230162,P178,7962,1,0
191,Internet Explorer,-11.297308,P178,1358,1,0


In [859]:
items = [x for x in json_data if x["case_id"] in set(chosen.case_id)]

prompts = []

for item in items:
    s = item["requested_rewrite"]["subject"]
    ps = item["generation_prompts"]

    ps = [p.replace(s, "") for p in ps]
    prompts.extend(ps)

    

In [878]:
unique_prompts = list(set(prompts))
# "recently entered" --> "entered" 

# speak --> spoke
# is --> was
# carries -> carried
# produces -> produced
def tense_to_past(prompt):
    p = (
        prompt
        .replace(" is", " was")
        .replace(" speak", " spoke")
        .replace(" carries", " carried")
        .replace(" produces", " produced")
        .replace(" include", " included")
        .replace(" works", " worked")
        .replace(" has", " had")
        .replace(" lives", " lived")
        .replace(" sells", " sold")
        .replace(" owns", " owned")
        .replace("recently entered", "entered")
        # .replace(" restaurants", " sights") # not about tense, should be its own thing
    )

    return(p)

unique_prompts

[' recently entered an agreement with the record label',
 "'s friends all speak the language of",
 "'s greatest strength is",
 'The language used by people in  is',
 'In , people speak',
 'The passport that  carries is',
 ' is my favorite show that has aired on',
 "One can get to 's headquarters by navigating",
 "The company that owns and sells 's music is",
 ' is known for',
 'The development of  is overseen by',
 ' is surrounded by restaurants including',
 "'s surroundings include",
 ' is full of people who speak',
 'The founders of  are from the location of',
 "'s greatest accomplishment is",
 "'s parents are from",
 'Where  passed away, people speak the language of',
 ' was born in',
 ' produces the most amazing music on the',
 ' currently lives in',
 ' is extraordinarily good at',
 "'s favorite lunchtime work meals include",
 'The tragic death of  occurred in',
 "'s work office is surrounded by",
 ' originates from',
 ' is sold by',
 "'s greatest artistic work is",
 'When  was kil

In [879]:
[tense_to_past(p) for p in unique_prompts]

[' entered an agreement with the record label',
 "'s friends all spoke the language of",
 "'s greatest strength was",
 'The language used by people in  was',
 'In , people spoke',
 'The passport that  carried was',
 ' was my favorite show that had aired on',
 "One can get to 's headquarters by navigating",
 "The company that owned and sold 's music was",
 ' was known for',
 'The development of  was overseen by',
 ' was surrounded by sights including',
 "'s surroundings included",
 ' was full of people who spoke',
 'The founders of  are from the location of',
 "'s greatest accomplishment was",
 "'s parents are from",
 'Where  passed away, people spoke the language of',
 ' was born in',
 ' produced the most amazing music on the',
 ' currently lived in',
 ' was extraordinarily good at',
 "'s favorite lunchtime work meals included",
 'The tragic death of  occurred in',
 "'s work office was surrounded by",
 ' originates from',
 ' was sold by',
 "'s greatest artistic work was",
 'When  was k

In [952]:
# write the thing to get over all the prompts

c = chosen.loc[0]
item = [x for x in json_data if x["case_id"]==c.case_id][0]

rewrite = item["requested_rewrite"]
gen_prompts = [x for x in item["generation_prompts"]] if c.past==0 else [tense_to_past(x) for x in item["generation_prompts"]]


In [953]:
rewrite

{'prompt': '{} writes in',
 'relation_id': 'P1412',
 'target_new': {'str': 'Norwegian', 'id': 'Q9043'},
 'target_true': {'str': 'English', 'id': 'Q1860'},
 'subject': 'Clark Gable'}

In [3]:
df = pd.read_csv("counterfact/counterfact-selected-relations.csv")

In [9]:
rel_ids = df.relation_id.to_list()

json_out = []

for j in json_data:
    if j["requested_rewrite"]["relation_id"] in rel_ids: json_out.append(j)

print(len(json_out))

17584


In [12]:
with open("counterfact/counterfact-selected.json", "w") as file:
    json.dump(json_out, file, indent=4)